In [ ]:
!pip install datasets

In [3]:
from datasets import load_dataset, Dataset
import openai
import csv
dataset1 = load_dataset('json', data_files='sub_datasets/sub_dataset_1.json', split=None)
dataset2 = load_dataset('json', data_files='sub_datasets/sub_dataset_2.json', split=None)
dataset3 = load_dataset('json', data_files='sub_datasets/sub_dataset_3.json', split=None)
dataset4 = load_dataset('json', data_files='sub_datasets/sub_dataset_4.json', split=None)
dataset5 = load_dataset('json', data_files='sub_datasets/sub_dataset_5.json', split=None)

c:\Users\HUAWEI\.conda\envs\llama\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 1
from openai import OpenAI

# 替换为您的 API 令牌
API_TOKEN = 'sk-4f65d8c5f266295f818dcf5712449fa3'

# 创建 OpenAI 客户端
client = OpenAI(
    api_key=API_TOKEN,
    base_url="https://api.atomecho.cn/v1",  # Llama 社区的 API 地址
)

def get_response(question, max_retries=5, wait_time=5):
    conversation_history = [{"role":"system","content":"你是一个圆锥曲线很厉害的数学家"}]
    for q in  question:
        conversation_history.append({"role": "user", "content":q})

        # 从模型中流式获取响应
        stream = client.chat.completions.create(
            model="Llama3-Chinese-8B-Instruct",
            messages=conversation_history,
            stream=True,
        )
        response = ""

        for chunk in stream:
            try:
                if chunk.choices[0].delta.content is not None:
                    response_chunk = chunk.choices[0].delta.content
                    response += response_chunk
            except IndexError as e:
                pass
        conversation_history.append({"role": "assistant", "content": response})
    
    return response

In [5]:
# 2
import time
batch_size = 10
total_requests = 500
ansall = []
start_time = time.time()

for i in range(500):
    if i % batch_size == 0 and i != 0:    
        elapsed_time = time.time() - start_time
        if elapsed_time < 50:
            time_to_wait = 50 - elapsed_time
            print(f"Waiting for {time_to_wait:.2f} seconds...")
            time.sleep(time_to_wait)
        start_time = time.time()  # 重置计时器
        
    print("-" * 20, i)
    item = dataset3['train'][i]
    print('问题', item['text'])
    print('正确答案', item['answer_expressions'])
    
    question = [item['text'],
                "请直接给出上面问题的答案，不需要分析过程和说明文字"]
    answer = get_response(question)
    print('llama答案', answer)

    ansall.append([i, answer, item['answer_expressions']])

-------------------- 0
问题 已知抛物线$x^{2}=4 y$上一点$M(x_{1}, y_{1})$到其焦点的距离为$3$，则点$M$到原点的距离?
正确答案 2*sqrt(3)
llama答案 点 M 到 原 点 的 距离 为 $15$.
-------------------- 1
问题 抛物线$y^{2}=-8 x$的准线方程是?
正确答案 x=2
llama答案 准线方程为 $x = 0$.
-------------------- 2
问题 已知双曲线$C$: $\frac{x^{2}}{6}-\frac{y^{2}}{3}=1$，则$C$的焦点到其渐近线的距离是?
正确答案 sqrt(3)
llama答案 抱歉，我无法给出答案，因为我不知道渐近线的具体方程。该问题中的渐近线未提及，也无从确定。
-------------------- 3
问题 已知$A(x_{1}, y_{1})$ ,$B(x_{2}, y_{2})$是抛物线$C$:$ y^{2}=4 x$上两点，线段$A B$的垂直平分线与$x$轴相交于点$D(m, 0)$，若$x_{1}+x_{2}=2$，则$m$=?
正确答案 3
llama答案 答案是 $-\frac{\sqrt{(b-a)^2}}{6}$。
-------------------- 4
问题 已知定点$A(4,0)$和圆$x^{2}+y^{2}=4$上的动点$B$，动点$P(x, y)$满足$\overrightarrow{O A}+\overrightarrow{O B}=2 \overrightarrow{O P}$，则点$P$的轨迹方程为?
正确答案 (x-2)^2+y^2=1
llama答案 $(x+y)(x-y)+(x+y)(x+y)-xy=4$
-------------------- 5
问题 双曲线$\frac{x^{2}}{4}+\frac{y^{2}}{k}=1$的一个焦点到其渐近线距离为$3$，则实数$k$的值为?
正确答案 -9
llama答案 对于这个问题，没有明确的唯一解，因此无法提供具体的值。
-------------------- 6
问题 已知直线$x=m$与双曲线$C$: $\frac{x^{2}}{a^{2}}-\frac{y^{2}}{b^{2}}=1(a>0

In [6]:
# 打开一个CSV文件进行写入
with open('llama7b_result3.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # 循环写入数据
    for row in ansall:
        writer.writerow(row)

print('数据已成功写入CSV文件.')

数据已成功写入CSV文件.


In [ ]:
!pip install zhipuai

In [9]:
from zhipuai import ZhipuAI
# 创建 OpenAI 客户端实例
# GPT-3.5-turbo
client = ZhipuAI(api_key="15bb7a5e8a7696d57023e25e5f9a47e2.0gEPh5r0RMNz0vIN")

def get_response(question, max_retries=5, wait_time=5):
    conversation_history = [{"role":"system","content":"you are a talented mathematician"}]
    for q in  question:
        conversation_history.append({"role": "user", "content":q})

        # 从模型中流式获取响应
        stream = client.chat.completions.create(
            model="glm-4",
            messages=conversation_history,
            stream=True
        )
        response = ""

        for chunk in stream:
            try:
                if chunk.choices[0].delta.content is not None:
                    response_chunk = chunk.choices[0].delta.content
                    response += response_chunk
            except IndexError as e:
                pass
        conversation_history.append({"role": "assistant", "content": response})
    
    return response
    
GPT35res = []

print('在测试集上检查GLM-4表现')
for i in list(range(500)):
    print('%'*30, i, '%'*30)
    item = dataset3['train'][i]
    print('问题', item['text'])
    print('正确答案', item['answer_expressions'])
    
    question = [item['text']+'请输出你的最终答案并保证这个答案是最简形式，不需要任何其他内容，只输出答案本身']
    answer = get_response(question)
    print('glm4答案', answer)
    GPT35res.append([item['text'], item['answer_expressions'], answer])
    

在测试集上检查GLM-4表现
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 0 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
问题 已知抛物线$x^{2}=4 y$上一点$M(x_{1}, y_{1})$到其焦点的距离为$3$，则点$M$到原点的距离?
正确答案 2*sqrt(3)
glm4答案 4
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 1 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
问题 抛物线$y^{2}=-8 x$的准线方程是?
正确答案 x=2
glm4答案 $$x=2$$
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 2 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
问题 已知双曲线$C$: $\frac{x^{2}}{6}-\frac{y^{2}}{3}=1$，则$C$的焦点到其渐近线的距离是?
正确答案 sqrt(3)
glm4答案 $\sqrt{3}$
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 3 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
问题 已知$A(x_{1}, y_{1})$ ,$B(x_{2}, y_{2})$是抛物线$C$:$ y^{2}=4 x$上两点，线段$A B$的垂直平分线与$x$轴相交于点$D(m, 0)$，若$x_{1}+x_{2}=2$，则$m$=?
正确答案 3
glm4答案 $m=2$
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 4 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
问题 已知定点$A(4,0)$和圆$x^{2}+y^{2}=4$上的动点$B$，动点$P(x, y)$满足$\overrightarrow{O A}+\overrightarrow{O B}=2 \overrightarrow{O P}$，则点$P$的轨迹方程为?
正确答案 (x-2)^2+y^2=1
glm4答案 $(x-2)^{2}+y^{2}=1$
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 5 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
问题 双曲线$\frac{x^{2}}{4}+\frac{y^{2}}{k}=1$的一个焦

In [10]:
# 打开一个CSV文件进行写入
with open('glm4_result3_short.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # 循环写入数据
    for row in GPT35res:
        writer.writerow(row)

print('数据已成功写入CSV文件.')

数据已成功写入CSV文件.
